In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seml
import pandas as pd
import json
from collections import defaultdict
from functools import reduce

In [2]:
#collection_name = 'week8_cora_full_ensemble'
collection_name = 'week8_cora_full_dropout'

collection = seml.database.get_collection(collection_name)
experiments = [{'config' : r['config'], 'result' : r['result'], 'id' : r['_id']} for r in collection.find() if r['status'] in ('COMPLETED',)]
for ex in experiments:
    with open(ex['result']) as f:
        ex['result'] = json.load(f)

print(f'Number of finished experiments : {len(experiments)}')

FileNotFoundError: [Errno 2] No such file or directory: '/nfs/students/fuchsgru/artifacts/week8_cora_full_dropout/gcn_ai_r:T_sn:T_ro:T/metrics.json'

In [ ]:
def get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True):
    exs = [
        r for r in experiments if r['config']['model']['residual'] == residual and r['config']['model']['use_spectral_norm'] == spectral_norm and r['config']['data']['train_labels_remove_other'] == train_labels_remove_other
    ]
    return exs[0]

In [ ]:
experiment = get_experiment(experiments, residual=True, spectral_norm=True, train_labels_remove_other=True)

In [ ]:
print([k for k in experiment['result'].keys() if 'auroc' in k])

In [ ]:

d = {}
for experiment in experiments:
    residual = experiment['config']['model']['residual']
    spectral_norm = experiment['config']['model']['use_spectral_norm']
    remove_other = experiment['config']['data']['train_labels_remove_other']
    if any('Operating_Systems' in label for label in experiment['config']['data']['val_labels']):
        ood = 'os'
    else:
        ood = 'ai'
    
    prefix = (ood, remove_other, residual, spectral_norm)
    
    for k, v in experiment['result'].items():
        mean, std = np.array(v).mean(), np.array(v).std()
        med = np.median(np.array(v))
        if 'auroc' in k:
            if 'no-edges' in k:
                no_edges = True
                k = k.replace('_no-edges', '')
                k = k.replace('-no-edges', '')
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            k, ood_type = '_'.join(k_tokens[:-1]), k_tokens[-1]
            prefix_edges = prefix + (no_edges, ood_type,)
                
            if not 'no' in k and ':' in k:
                continue
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'mean')] = [mean]
            d[prefix_edges + ('AUC-ROC', k.replace('auroc_', '').replace('-', ' '), 'std')] = [std]
            #d[prefix_edges + ('feature density', k.replace('auroc:', ''), 'median')] = [med]
        elif k.startswith('accuracy'):
            k_tokens = k.split('_')
            if k_tokens[1] in ('id', 'ood'):
                k_tokens = ['_'.join(k_tokens[:2])] + k_tokens[2:]
            if 'no-edges' in k_tokens[-1]:
                k_tokens[-1] = k_tokens[-1].replace('no-edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            k_tokens = [token for token in k_tokens if len(token) > 0]
            if len(k_tokens) == 3:
                ood_type = k_tokens[-1]
                k_tokens = k_tokens[:-1]
            else:
                #ood_type = ''
                continue
            name, dataset = k_tokens
            
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'mean')] = [mean]
            d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'std')] = [std]
            #d[prefix + (no_edges, ood_type, 'accuracy', name.replace('accuracy', '').replace('_', ''), 'med')] = [med]
        elif k.startswith('ece') and False:
            if 'no_edges' in k:
                k = k.replace('no_edges', '')[:-1]
                no_edges = True
            else:
                no_edges = False
            
            k_tokens = k.split('_')
            d[prefix + (no_edges, '', 'ece', '', 'mean')] = [mean]
            d[prefix + (no_edges, '', 'ece', '', 'std')] = [std]
            
            #print(name, dataset, prefix + (no_edges, ood_type,))
            
            
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'mean')] = [mean]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'std')] = [std]
            #d[(ood, remove_other, residual, spectral_norm, '', k, '', 'median')] = [med]
                
df = pd.DataFrame(d).T.sort_index()

In [26]:
df

0
ai False False False False bernoulli AUC-ROC  expected softmax entropy mean  0.65
                                                                       std   0.08
                                              logit energy             mean  0.66
                                                                       std   0.07
                                              max score                mean  0.63
...                                                                           ...
os True  True  True  True  normal    accuracy                          std   0.04
                                              id                       mean  0.75
                                                                       std   0.04
                                              ood                      mean  0.14
                                                                       std   0.09

[1728 rows x 1 columns]

In [27]:
df = df.reset_index((0, 1, 2, 3, 5, -1))

In [28]:
df.shape

(1728, 7)

In [29]:
df

level_0  level_1  level_2  level_3  \
False AUC-ROC  expected softmax entropy      ai    False    False    False   
               expected softmax entropy      ai    False    False    False   
               logit energy                  ai    False    False    False   
               logit energy                  ai    False    False    False   
               max score                     ai    False    False    False   
...                                         ...      ...      ...      ...   
True  accuracy                               os     True     True     True   
               id                            os     True     True     True   
               id                            os     True     True     True   
               ood                           os     True     True     True   
               ood                           os     True     True     True   

                                           level_5 level_8     0  
False AUC-ROC  expected softmax entropy  bernoulli    mean  0.65  
               expected softmax entropy  bernoulli     std  0.08  
               logit energy              bernoulli    mean  0.66  
               logit energy              bernoulli     std  0.07  
               max score                 bernoulli    mean  0.63  
...                                            ...     ...   ...  
True  accuracy                              normal     std  0.04  
               id                           normal    mean  0.75  
               id                           normal     std  0.04  
               ood                          normal    mean  0.14  
               ood                          normal     std  0.09  

[1728 rows x 7 columns]

In [30]:
subdfs, names = [], []
for n, g in df.groupby(['level_0', 'level_1', 'level_2', 'level_3', 'level_5', 'level_8']):
    g = pd.DataFrame(g[0])
    g.columns = [n]
    subdfs.append(g)

In [31]:
df_cat = pd.concat(subdfs, axis=1)
idx = pd.MultiIndex.from_tuples(df_cat.columns, names=('OOD Classes', 'Remove OOD', 'Residual', 'Spectral Norm', 'OOD Experiment', 'Stat'))
df_cat.columns = idx
df_cat.index.names = ('Remove-Edges', '', '')
pd.set_option("display.precision", 2)
df_cat = df_cat.T.sort_index().T

In [32]:
exp_sizes = [
    1, # ood data
    1, # remove ood vertices from train
    2, # residual,
    2, # Spectral norm,
    3, # ood type
    2, # stats
]
exp_size = reduce(lambda x, y: x * y, exp_sizes)
exp_size

24

In [35]:


df_cat.iloc[:, 0 * exp_size : 1 * exp_size : 2]

OOD Classes                                           ai               \
Remove OOD                                         False                
Residual                                           False                
Spectral Norm                                      False                
OOD Experiment                                 bernoulli   loc normal   
Stat                                                mean  mean   mean   
Remove-Edges                                                            
False        AUC-ROC  expected softmax entropy      0.65  0.85   0.32   
                      logit energy                  0.66  0.86   0.12   
                      max score                     0.63  0.83   0.59   
                      mutual information            0.60  0.67   0.82   
                      predicted class variance      0.60  0.58   0.82   
                      total predictive entropy      0.65  0.84   0.51   
             accuracy                               0.80  0.81   0.75   
                      id                            0.81  0.78   0.80   
                      ood                           0.73   NaN   0.35   
True         AUC-ROC  expected softmax entropy      1.00  0.80   0.01   
                      logit energy                  1.00  0.78   0.00   
                      max score                     1.00  0.78   0.50   
                      mutual information            0.29  0.68   0.80   
                      predicted class variance      0.15  0.58   0.85   
                      total predictive entropy      1.00  0.79   0.35   
             accuracy                               0.72  0.78   0.70   
                      id                            0.78  0.74   0.77   
                      ood                           0.16   NaN   0.12   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.64  0.84  1.83e-01   
                      logit energy                  0.64  0.82  1.72e-01   
                      max score                     0.64  0.88  3.39e-01   
                      mutual information            0.47  0.43  8.00e-01   
                      predicted class variance      0.43  0.28  7.86e-01   
                      total predictive entropy      0.65  0.88  2.31e-01   
             accuracy                               0.80  0.81  7.62e-01   
                      id                            0.81  0.78  8.05e-01   
                      ood                           0.75   NaN  3.80e-01   
True         AUC-ROC  expected softmax entropy      0.99  0.84  4.76e-04   
                      logit energy                  0.95  0.80  9.07e-05   
                      max score                     0.97  0.84  8.50e-02   
                      mutual information            0.04  0.32  9.95e-01   
                      predicted class variance      0.09  0.25  9.92e-01   
                      total predictive entropy      0.98  0.85  1.14e-02   
             accuracy                               0.71  0.74  6.84e-01   
                      id                            0.76  0.72  7.47e-01   
                      ood                           0.18   NaN  1.20e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                

In [36]:

df_cat.iloc[:, 1 * exp_size : 2 * exp_size : 2]

OOD Classes                                           ai                  \
Remove OOD                                          True                   
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.68  0.91  3.36e-01   
                      logit energy                  0.68  0.90  1.16e-01   
                      max score                     0.66  0.89  6.38e-01   
                      mutual information            0.62  0.72  8.18e-01   
                      predicted class variance      0.64  0.61  8.31e-01   
                      total predictive entropy      0.68  0.91  5.56e-01   
             accuracy                               0.82  0.82  7.75e-01   
                      id                            0.82  0.84  8.23e-01   
                      ood                           0.80   NaN  3.49e-01   
True         AUC-ROC  expected softmax entropy      1.00  0.86  1.75e-02   
                      logit energy                  1.00  0.86  1.27e-03   
                      max score                     1.00  0.85  5.39e-01   
                      mutual information            0.30  0.74  8.27e-01   
                      predicted class variance      0.17  0.59  8.65e-01   
                      total predictive entropy      1.00  0.86  3.98e-01   
             accuracy                               0.72  0.78  7.20e-01   
                      id                            0.78  0.77  7.84e-01   
                      ood                           0.15   NaN  1.43e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.68  0.84  1.54e-01   
                      logit energy                  0.66  0.81  1.51e-01   
                      max score                     0.67  0.89  3.60e-01   
                      mutual information            0.45  0.44  8.71e-01   
                      predicted class variance      0.41  0.27  8.38e-01   
                      total predictive entropy      0.69  0.89  2.35e-01   
             accuracy                               0.82  0.82  7.79e-01   
                      id                            0.83  0.82  8.19e-01   
                      ood                           0.77   NaN  4.14e-01   
True         AUC-ROC  expected softmax entropy      0.99  0.85  4.54e-05   
                      logit energy                  0.94  0.81  0.00e+00   
                      max score                     0.97  0.87  9.79e-02   
                      mutual information            0.04  0.32  9.98e-01   
                      predicted class variance      0.08  0.24  9.96e-01   
                      total predictive entropy      0.98  0.87  1.22e-02   
             accuracy                               0.71  0.75  7.02e-01   
                      id                            0.77  0.75  7.61e-01   
                      ood                           0.15   NaN  1.71e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                      

In [37]:

df_cat.iloc[:, 2 * exp_size : 3 * exp_size : 2]

OOD Classes                                           os               \
Remove OOD                                         False                
Residual                                           False                
Spectral Norm                                      False                
OOD Experiment                                 bernoulli   loc normal   
Stat                                                mean  mean   mean   
Remove-Edges                                                            
False        AUC-ROC  expected softmax entropy      0.64  0.94   0.30   
                      logit energy                  0.65  0.93   0.12   
                      max score                     0.62  0.91   0.58   
                      mutual information            0.58  0.61   0.79   
                      predicted class variance      0.58  0.50   0.80   
                      total predictive entropy      0.64  0.93   0.49   
             accuracy                               0.80  0.81   0.75   
                      id                            0.81  0.78   0.80   
                      ood                           0.73   NaN   0.35   
True         AUC-ROC  expected softmax entropy      1.00  0.85   0.01   
                      logit energy                  1.00  0.86   0.00   
                      max score                     1.00  0.84   0.49   
                      mutual information            0.30  0.73   0.80   
                      predicted class variance      0.17  0.59   0.85   
                      total predictive entropy      1.00  0.86   0.37   
             accuracy                               0.72  0.78   0.70   
                      id                            0.78  0.74   0.77   
                      ood                           0.16   NaN   0.12   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.64  0.91  1.75e-01   
                      logit energy                  0.63  0.88  1.79e-01   
                      max score                     0.63  0.95  3.35e-01   
                      mutual information            0.45  0.33  8.09e-01   
                      predicted class variance      0.41  0.20  7.88e-01   
                      total predictive entropy      0.64  0.96  2.28e-01   
             accuracy                               0.80  0.81  7.61e-01   
                      id                            0.81  0.78  8.03e-01   
                      ood                           0.74   NaN  3.91e-01   
True         AUC-ROC  expected softmax entropy      0.99  0.88  1.59e-04   
                      logit energy                  0.94  0.83  2.27e-05   
                      max score                     0.97  0.89  8.23e-02   
                      mutual information            0.05  0.31  9.91e-01   
                      predicted class variance      0.08  0.23  9.87e-01   
                      total predictive entropy      0.98  0.90  1.28e-02   
             accuracy                               0.70  0.74  6.88e-01   
                      id                            0.76  0.72  7.50e-01   
                      ood                           0.17   NaN  1.37e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                           True                    
Spectral Norm                                

In [38]:

df_cat.iloc[:, 3 * exp_size : 4 * exp_size : 2]

OOD Classes                                           os                  \
Remove OOD                                          True                   
Residual                                           False                   
Spectral Norm                                      False                   
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.68  0.95  3.32e-01   
                      logit energy                  0.66  0.94  1.14e-01   
                      max score                     0.65  0.93  6.43e-01   
                      mutual information            0.62  0.63  8.35e-01   
                      predicted class variance      0.63  0.53  8.48e-01   
                      total predictive entropy      0.67  0.94  5.63e-01   
             accuracy                               0.82  0.82  7.75e-01   
                      id                            0.82  0.84  8.23e-01   
                      ood                           0.80   NaN  3.49e-01   
True         AUC-ROC  expected softmax entropy      1.00  0.89  1.81e-02   
                      logit energy                  1.00  0.87  3.40e-04   
                      max score                     1.00  0.87  5.41e-01   
                      mutual information            0.31  0.78  8.21e-01   
                      predicted class variance      0.18  0.60  8.64e-01   
                      total predictive entropy      1.00  0.89  4.09e-01   
             accuracy                               0.72  0.78  7.20e-01   
                      id                            0.78  0.77  7.84e-01   
                      ood                           0.15   NaN  1.43e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                                                   
Spectral Norm                                      True                    
OOD Experiment                                 bernoulli   loc    normal   
Stat                                                mean  mean      mean   
Remove-Edges                                                               
False        AUC-ROC  expected softmax entropy      0.68  0.90  1.64e-01   
                      logit energy                  0.65  0.86  1.50e-01   
                      max score                     0.67  0.93  3.79e-01   
                      mutual information            0.45  0.30  8.73e-01   
                      predicted class variance      0.41  0.18  8.43e-01   
                      total predictive entropy      0.68  0.92  2.48e-01   
             accuracy                               0.82  0.82  7.80e-01   
                      id                            0.83  0.83  8.20e-01   
                      ood                           0.77   NaN  4.20e-01   
True         AUC-ROC  expected softmax entropy      0.99  0.89  9.07e-05   
                      logit energy                  0.94  0.83  2.27e-05   
                      max score                     0.97  0.90  1.03e-01   
                      mutual information            0.04  0.31  9.93e-01   
                      predicted class variance      0.07  0.23  9.92e-01   
                      total predictive entropy      0.98  0.91  1.54e-02   
             accuracy                               0.71  0.76  7.01e-01   
                      id                            0.77  0.75  7.61e-01   
                      ood                           0.13   NaN  1.63e-01   

OOD Classes                                                               \
Remove OOD                                                                 
Residual                                      